In [1]:
import qutip, pickle, sys
import matplotlib.pyplot as plt 
import numpy as np
import scipy.optimize as opt 
import scipy.linalg as linalg
import time as time
import math, cmath
import Max_Ent_aux_lib as me
#import proj_ev_library as projev
#import max_entev library as meev
from IPython.display import display, Math, Latex

np.set_printoptions(threshold=1.e-3,linewidth=120,precision=1, formatter={"float":lambda x: str(.001*int(1000*x)) })

Tenemos la siguiente ecuación de movimiento:

$$
    \dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{\bigg(c_{M+2}(t) - c_M(t) \bigg)\langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle + \bigg(c_{M-2}(t) - c_M(t)\bigg) \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle + p \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle \bigg\}
$$

siendo $I_{M}^{0}(t) = \langle \rho_{M}^{0\dagger}(t) \rho_{M}^{0}(t)\rangle = I_{-M}^{0}(t)$
la cual puede reescribirse como 

$$
\dot{c}_M(t) I_{M}^{0}(t) = -i \bigg\{A_{\rho^{M, M+2}} c_{M+2}(t) - \bigg(A_{\rho^{M, M+2}} + B_{\rho^{M, M-2}} \bigg) c_{M}(t) + B_{\rho^{M, M-2}} c_{M-2}(t) \bigg\} + p C_{\rho^M} \\
= - i({\bf{\mathcal{M}}}(t) \textbf{c}(t))_M
$$

donde 

<ol>

<li> $A_{\rho^{M, M+2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,-2}, \rho_{M+2}^0] \rangle$ </li> 
<li> $B_{\rho^{M, M-2}} = \langle \rho_{M}^{0\dagger}(t) [H_{0,2}, \rho_{M-2}^0] \rangle$ </li>
<li> $C_{\rho^M} = \langle \rho_{M}^{0\dagger}(t) [\Sigma, \rho_{M}^0 (t)]\rangle$ </li>
    
</ol>    

Entonces, si $f(t) = \langle \rho(t) \rho^0(t) \rangle = \sum_{M} \dot{c}_M(t) I_{M}^{0}(t)$

$$
    \dot{f}(t) = \sum_{M} \bigg(\dot{c}_M(t) I_{M}^{0}(t) + {c}_M(t) \dot{I}_{M}^{0}(t) \bigg)
$$

where

1: $A=\alpha B = N(t) \exp(-m t^{1+a}) = I_M^{(0)}$ (a primer orden alpha = 1) \
2: $C = M N(t) exp(-m t^{1+a})$

con $N(t) = 1/Tr(\rho(t))$ la calculo con la definición del $\rho_M (t)=  c_{M}(t) \rho^0_M(t)$, with $c_0(t)$= 1. 
Note that the kernel $K(t,t') = e^{(-i M(t-t'))}$ is not a solution to the previous differential equation for the M-tensor is time-dependent
    
$H_ {0, 2}^\dagger = H_{0, -2}$ 

$\rho_{M}^\dagger = \rho_{-M}$

## Step 1: Fix parameters and initial conditions for the coherences

In [2]:
p = .0009;    # strength of the Sigma Interaction Hamiltonian
a = -.05;      # Power-law factor
M = 3;        # Truncation/Total no. of coherences
coherences_t0_pert0 = 1. # Unused for the time being
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a} # dictionary containing the simulation's initial 
                                                                         # parameters 

cohr_complex_t0 = [coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])] 
    # initial configuration of complex-valued coherences, random numbers for the time being. Can be changed in the future

cohr_complex_t0 += [0 for i in range(param_list["total_no_cohrs"])]
    # normalization of the initial vector, so that Tr c_0 = 1. 
cohr_t0_trace = sum(cohr_complex_t0)
cohr_complex_t0 = [cohr/cohr_t0_trace for cohr in cohr_complex_t0]
cohr_complex_t0 = np.array(cohr_complex_t0)

Since the coherences are complex-valued numbers, the previous system of $M \times M$ coupled complex-valued differential equations can be rewritten as a system of $2M \times 2M$ coupled real-valued differential equations, as follows:

if $c_M(t) = a_M(t) + i b_M(t)$, then 

$$
    \dot{a}_M(t) + i \dot{b}_M(t) = -i \sum_{m'}{\bf{\mathcal{M}}}_{Mm'}(t) \bigg(a_{m'}(t) + i b_{m'}(t)\bigg),
$$

$$
    \dot{a}_M(t) + i \dot{b}_M(t) = -i \sum_{m'}{\bf{\mathcal{M}}}_{Mm'}(t) a_{m'}(t) + \sum_{m'} {\bf{\mathcal{M}}}_{Mm'}(t) b_{m'}(t)
$$

$$
    \dot{{\bf a}}(t) = {\cal M}(t) \cdot {\bf b}(t), \quad \dot{{\bf b}}(t) = -{\cal M}(t) \cdot {\bf a}(t), \qquad s.t. \quad{\bf c}(t) = {\bf a}(t) + i {\bf b}(t)
$$


## Step 2: Setting up and Solving system of complex diff. eqs

In [3]:
param_list = {"total_no_cohrs": M, "p_factor": p, "power_law_factor": a} # dictionary containing the simulation's initial 
                                                                         # parameters 

cohr_complex_t0 = [coherences_t0_pert0 - np.random.rand() for i in range(param_list["total_no_cohrs"])]
cohr_complex_t0 += [0 for i in range(param_list["total_no_cohrs"])]
cohr_complex_t0 = np.array(cohr_complex_t0)
param_list = [M, p, a]

In [4]:
B = me.complex_differential_system(parameters = param_list, cohr_complex = cohr_complex_t0, t=1)
len(B)

6

In [46]:
from scipy.integrate import odeint
ts = np.linspace(0.1, 1., 100)              ## times 
time_ev_ReIm_cohrs = odeint(func = me.complex_differential_system, 
                y0 = cohr_complex_t0, 
                t = ts,
                args = ((param_list,)))
#  tiempo vertical, primera linea es la config init: t  cohr
time_ev_ReIm_cohrs

array([[0.20500000000000002, 0.01, 0.251, 0.0, 0.0, 0.0],
       [0.20500000000000002, 0.01, 0.251, 0.0, 0.003, 0.001],
       [0.20500000000000002, 0.01, 0.251, 0.0, 0.006, 0.002],
       ...,
       [0.325, 0.032, 0.34600000000000003, 0.04, 0.322, 0.132],
       [0.328, 0.033, 0.34800000000000003, 0.041, 0.326, 0.133],
       [0.331, 0.033, 0.35000000000000003, 0.042, 0.33, 0.135]])

In [47]:
len(time_evolved_real_imag_parts_cohrs) 

100

In [48]:
len(time_evolved_real_imag_parts_cohrs[1])

6

## Step 3: Obtaining the time-evolved complex-valued coherences

In [49]:
cohr1_t = [time_ev_ReIm_cohrs[t][0] + 1j * time_ev_ReIm_cohrs[t][0 + M] for t in range(100)]

In [54]:
vec_cohr_t = [[time_ev_ReIm_cohrs[t][m] + 1j * time_ev_ReIm_cohrs[t][m + M] for t in range(100)] for m in range(M)]
# prime parámetro coherencia
# segundo parámetro tiempo

In [56]:
fidelity_t0 = []

NameError: name 'fidelity_vs_t' is not defined

In [55]:
fidelity_vs_t = [[sum(result[:, m][int(list(ts).index(t)) 
                                   * -m * (a+1) * t**a / (me.A_mmplustwo_matrix_elmt(m, t, a))] for m in range(M))] for t in list(ts)]

NameError: name 'result' is not defined

In [72]:
len(result[:, 0])

100

In [73]:
len(result[0])

6

# Tests y fallos :(

In [23]:
Preguntas:
    
    1. La matrix M tiene que depender con el tiempo me parece, si A, B y C lo hacen: Check 
    2. Como tendría que implementar la norma???: Check
    3. Empezar a jugar con los parámetros: Checkn't
    
m+2 vs m+2 : Check
** reescribir la matriz en tèrminos de los c_pares. : Check

SyntaxError: invalid syntax (3991330021.py, line 1)